# California Yearly Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972
FREQUENCY = "yearly"

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

In [4]:
metadata = pd.read_csv("data/new_features_t2m/california/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods
0,c58d6cd4,predictor_c58d6cd4.parquet,0,5,NaN,1,yearly,0,ps
1,f6fe2bdd,predictor_f6fe2bdd.parquet,0,10,NaN,1,yearly,0,ps
2,f7e8e8b5,predictor_f7e8e8b5.parquet,0,15,NaN,1,yearly,0,ps
3,a2ab7bdb,predictor_a2ab7bdb.parquet,0,20,NaN,1,yearly,0,ps
4,bb11a417,predictor_bb11a417.parquet,0,30,NaN,1,yearly,0,ps
5,2deda7df,predictor_2deda7df.parquet,0,5,0.05,1,yearly,0,ps
6,d5a2e728,predictor_d5a2e728.parquet,0,10,0.05,1,yearly,0,ps
7,c8c5db88,predictor_c8c5db88.parquet,0,15,0.05,1,yearly,0,ps
8,3dd6f774,predictor_3dd6f774.parquet,0,20,0.05,1,yearly,0,ps
9,7a30744e,predictor_7a30744e.parquet,0,30,0.05,1,yearly,0,ps


In [5]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
region = "california"

In [6]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {0: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}.parquet")}
    experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("cv_r2", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results.csv")

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warn

# Chile Yearly Prediction

In [7]:
region="chile"

In [8]:
metadata = pd.read_csv(f"data/new_features_t2m/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods
0,a2b98342,predictor_a2b98342.parquet,0,5,NaN,1,yearly,0,ps
1,e1cb294f,predictor_e1cb294f.parquet,0,10,NaN,1,yearly,0,ps
2,72a2f71f,predictor_72a2f71f.parquet,0,15,NaN,1,yearly,0,ps
3,c8aa0653,predictor_c8aa0653.parquet,0,20,NaN,1,yearly,0,ps
4,c0bf31b9,predictor_c0bf31b9.parquet,0,30,NaN,1,yearly,0,ps
5,109982fa,predictor_109982fa.parquet,0,5,0.05,1,yearly,0,ps
6,47b0383e,predictor_47b0383e.parquet,0,10,0.05,1,yearly,0,ps
7,d885f1d7,predictor_d885f1d7.parquet,0,15,0.05,1,yearly,0,ps
8,7c8ce27d,predictor_7c8ce27d.parquet,0,20,0.05,1,yearly,0,ps
9,370a2f52,predictor_370a2f52.parquet,0,30,0.05,1,yearly,0,ps


In [9]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [10]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {0: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}.parquet")}
    experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("cv_r2", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results.csv")

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\gaussian_process\kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warn